In [73]:
import os
import os.path
from datetime import datetime

import pandas as pd
from pandas.errors import EmptyDataError
from sqlalchemy import create_engine, text

pd.options.mode.chained_assignment = None


In [74]:
BASE_DIR = '../../../../workspace/HelloPython/HistoricalMarketData/TechnicalIndicators'
TABLE_EQUITIES_DATA = 'equities_historic_data'
DURATIONS = (14, 30, 90, 200)  # Roughly for bi-weekly, monthly, quarterly, and 200 days running averages

In [75]:
try:
    DB = os.environ["DB"]
    DB_USER = os.environ["DB_USER"]
    DB_PWD = os.environ["DB_PWD"]
except KeyError:
    raise Exception("Required environment variables DB_USER and DB_PWD not set")
DB_URL = 'mysql+mysqlconnector://' + DB_USER + ':' + DB_PWD + '@localhost/' + DB
ENGINE = create_engine(DB_URL)

In [76]:
def generate_file_path(symbol, date=None):
    """
    Generates a file path for a given symbol
    to retrieve calculated tech indicator data
    from local CSV records
    :param symbol: ticker
    :param date: date embedded in the file name
    :return: file name and path
    """
    if date is not None:
        str_date = datetime.strftime(date, '%Y%m%d')
        file_name = symbol.lower() + '_' + str_date + '.csv'
    else:
        file_name = symbol.lower() + '.csv'
    file_path = os.path.join(BASE_DIR, file_name)
    if file_path is None:
        print('Could not find file for symbol:{}'.format(symbol))
    # print(file_path)
    return file_path, file_name

In [77]:
# def retrieve_technical_indicator_data_for_symbol_old(symbol):
#     """
#     Retrieves data from SQL. Superseded as we are now
#     retrieving historical data from CSV
#     :param symbol:
#     :return:
#     """
#     dt_last_entry = None
#     with ENGINE.connect() as conn:
#         res = conn.execute(text('select * from equities_historic_data where \
#                     symbol like \'' + symbol + '\''))
#     dfrm_existing = pd.DataFrame(res.mappings().all())
#     if dfrm_existing is not None and len(dfrm_existing) > 0:  # Don't do anything if nothing exists for symbol
#         dfrm_existing.set_index('date', inplace=True)
#         dfrm_existing.drop(columns=['netChange', 'pcntChange', 'open', 'high', 'low', 'symbol'], inplace=True)
#         reqd_columns = ['close', 'volume']
#         dfrm_existing.columns = [
#             symbol.upper()+'_'+column
#             for column in dfrm_existing.columns
#             if column in reqd_columns
#         ]
#         dfrm_existing.sort_index(inplace=True, ascending=True)
#     else:
#         print(f"No technical indicators data in DB for '{symbol}'.")
#     return dfrm_existing
#
# tmp_df = retrieve_technical_indicator_data_for_symbol_old('AAPL')
# tmp_df.tail(10)

In [78]:
def retrieve_technical_indicator_data_for_symbol(symbol, columns = None, feature=None)\
        -> pd.DataFrame:
    """
    Retrieves Technical Indicator Data for a given symbol
    for given columns and labels them as Y (dependent) or
    X (input features).
    :param symbol: Symbol for which to retrieve Technical Indicator Data
    :param columns: Subset of Technical Indicator Data to retrieve
    :param feature: Y or X_i label where 'i' is feature number
    :return:
    """
    file_path, _ = generate_file_path(symbol)
    if file_path is not None:
        try:
            dfrm = pd.read_csv(file_path)
            dfrm['date'] = pd.to_datetime(dfrm['date'])
            dfrm.set_index('date', inplace=True)
            dfrm.sort_index(ascending=True)
            dfrm.index.name = 'date'
            if feature is None:
                feature = symbol.upper()
            if columns is not None:
                dfrm = dfrm[columns]
                dfrm.columns = [
                feature.upper()+'_'+column
                for column in dfrm.columns
                if column in columns
                ]
            else:
                dfrm.columns = [
                feature.upper()+'_'+column
                for column in dfrm.columns
                ]
            return dfrm
        except FileNotFoundError as e:
            print('Exception reading input data for symbol {}. Generating metadata starting from baseline date.'.format(symbol.upper()))
            return None
        except EmptyDataError as e:
            print(f'No technical indicators found for {symbol.upper()}. Generating metadata starting from baseline date.')
            print('Generating all records.')
            return None
    else:
        print(f"No technical indicators data in DB for '{symbol}'.")
    return None

tmp_df = retrieve_technical_indicator_data_for_symbol('AAPL', ['volume', 'close'])
tmp_df.tail(10)

,AAPL_volume,AAPL_close
date,,
2025-04-16,59732423,194.27
2025-04-17,52164675,196.98
2025-04-21,46742537,193.16
2025-04-22,52976371,199.74
2025-04-23,52929165,204.60
2025-04-24,47310989,208.37
2025-04-25,38222258,209.28
2025-04-28,37626816,210.14
2025-04-29,36827633,211.21


In [79]:
def find_symbols_w_highest_correlations(symbol, correlations, count):
    """
    Finds tickets whose return have highest correlation
    with the returns of given symbol
    :param symbol:
    :param correlations:
    :param count:
    :return:
    """
    if len(correlations[correlations['level_0'] == symbol.upper()+'_close']) <= 0:
        print("No correlations found for symbol:{}.".format(symbol.upper()))
        print("Likely data does not go far enough back.")
        return None
    corrs = correlations[correlations['level_0'] == symbol.upper()+'_close']
    corrs.sort_values(0, ascending = False, inplace = True)
    return_symbols = [symbol.replace('_close', '') for symbol in corrs.loc[:,'level_1'].tolist()]
    return return_symbols[0:count]

In [80]:
def gen_corrs(dfrm):
    """
    Generate correlations numbers for entire
    input dataframe consisting of daily close values
    :param dfrm:
    :return:
    """
    correlations = dfrm[dfrm.columns].corr().abs().unstack().sort_values(kind="quicksort").reset_index()
    correlations = correlations[correlations['level_0'] != correlations['level_1']]
    #print(correlations)`
    return correlations

In [81]:
from functools import reduce
if __name__ == "__main__":
    MIN_ROW_COUNT = 5000 # Number of days to retrieve training data for
    COMP_COUNT = 10 # Number of stocks to find high correlations with

    query = 'SELECT symbol FROM equities_historic_data GROUP BY symbol HAVING COUNT(*) > '+str(MIN_ROW_COUNT)+''
    with ENGINE.connect() as conn:
        res = conn.execute(text(query))
    dfrm_symbols_list = pd.DataFrame(res.mappings().all())
    symbols = dfrm_symbols_list['symbol'].tolist() # this is out universe of symbol tickers

    # First calculate universal correlations
    dfrm_list_daily_closes = list()
    for symbol in symbols:
        dfrm = retrieve_technical_indicator_data_for_symbol(symbol, ['close', 'volume'])
        dfrm_list_daily_closes.append(dfrm)
    merged_dfrm_daily_close_values = reduce(lambda left, right: pd.merge(left, right, on='date'), dfrm_list_daily_closes)
    correlations = gen_corrs(merged_dfrm_daily_close_values)

    # Now merge a symbol's returns with returns of symbols
    # it has the highest correlations with. Then stack (concat)
    # all these returns across symbols to come up with our
    # training dataset

    # Data for a symbol and its corrs matches
    # Think of this as a single line in a matrix
    dfrm_list_single_row_data = list()
    # Data for all symbol and their corrs matches
    # Think of this as a matrix
    dfrm_list_full_matrix_data = list() # Data for
    tgt_symbols_list = ['JPM', 'C', 'MS', 'GS', 'WFC', 'BAC'] # Generate training data for this set only
    for symbol in tgt_symbols_list:
        # Our dependent variable Y - generalize the name so we can stack up many symbols for training dataset
        dfrm_symbol = retrieve_technical_indicator_data_for_symbol(symbol, None, 'Y')
        dfrm_list_single_row_data.append(dfrm_symbol)
        symbols_w_highest_corrs = find_symbols_w_highest_correlations(symbol, correlations, COMP_COUNT)
        if symbols_w_highest_corrs is None:
            continue # Likely not enough data for a symbol. Continue with rest
        print(f'Symbols with highest correlations with {symbol.upper()} are {symbols_w_highest_corrs}')
        counter = 1
        for comp_symbol in symbols_w_highest_corrs:
            feature = 'X_'+str(counter)
            dfrm_tmp = retrieve_technical_indicator_data_for_symbol(comp_symbol, None, feature)
            dfrm_list_single_row_data.append(dfrm_tmp)
            counter += 1
        # Now merge the symbol and all its corrs data against the same date
        merged_df = reduce(lambda left, right: pd.merge(left, right, on='date'), dfrm_list_single_row_data)
        dfrm_list_full_matrix_data.append(merged_df)
        dfrm_list_single_row_data.clear()
    # Now concatenate data. This means dates are repeated
    dfrm_aggr_training_data = pd.concat(dfrm_list_full_matrix_data)

Symbols with highest correlations with JPM are ['TXN', 'PNC', 'MAR', 'SPGI', 'SCHW', 'SIVB', 'SYK', 'WM', 'RJF', 'SNPS']
Symbols with highest correlations with C are ['AIG', 'RF', 'KEY', 'ZION', 'HIG', 'HBAN', 'ARNC', 'FITB', 'KIM', 'XRX']
Symbols with highest correlations with MS are ['HIG', 'DRE', 'ZION', 'KEY', 'FITB', 'LNC', 'MGM', 'RF', 'XRX', 'MET']
Symbols with highest correlations with GS are ['BK', 'STT', 'PRU', 'NTRS', 'CMA', 'CME', 'LNC', 'CBRE', 'SCHW', 'PFG']
Symbols with highest correlations with WFC are ['USB', 'DIS', 'PPG', 'UPS', 'SNA', 'IPG', 'OMC', 'MDLZ', 'AMGN', 'WHR']
No correlations found for symbol:BAC.
Likely data does not go far enough back.


In [82]:
dfrm_aggr_training_data.tail(10)

,Y_symbol,Y_close,Y_volume,Y_mean_200,Y_stddev_200,Y_pcntleStdDevs_200,Y_pcntleVolume_200,Y_pcntleClosing_200,Y_oscillator_200,Y_accu_dist_200,...,X_10_stddev_90,X_10_accu_dist_200,X_10_bollingerLower_200,X_10_bollingerUpper_200,X_10_mean_200,X_10_oscillator_200,X_10_pcntleClosing_200,X_10_pcntleStdDevs_200,X_10_pcntleVolume_200,X_10_stddev_200
date,,,,,,,,,,,,,,,,,,,,,
2025-04-16,WFC,63.98,17350992,70.867007,6.371775,33.576642,91.970803,15.328467,34.039334,-2.322492e+06,...,15.035748,-532056.138686,80.782134,131.331442,106.056788,0.000000,0.729927,100.0,89.781022,12.637327
2025-04-17,WFC,64.71,18961771,70.822391,6.370076,33.333333,92.028986,21.739130,36.800303,-1.943089e+06,...,14.801335,-508267.253181,80.254558,131.464573,105.859565,2.600897,2.173913,100.0,89.855072,12.802504
2025-04-21,WFC,64.01,17859094,71.105294,6.030890,30.147059,91.176471,13.970588,34.152799,-2.381621e+06,...,14.520378,-543653.691176,79.375332,131.897756,105.636544,0.000000,0.735294,100.0,91.911765,13.130606
2025-04-22,WFC,66.23,17536579,71.188015,5.883425,28.676471,90.441176,25.735294,37.898610,-1.845264e+06,...,14.101439,-546196.554071,78.727994,132.140536,105.434265,0.499109,1.470588,100.0,94.852941,13.353136
2025-04-23,WFC,67.94,20678780,71.268750,5.760794,27.205882,91.911765,27.941176,44.435284,-8.624221e+05,...,13.599921,-546637.727928,78.115036,132.364523,105.239779,1.247772,3.676471,100.0,96.323529,13.562372
2025-04-24,WFC,69.56,18420882,71.256277,5.741431,27.007299,90.510949,31.386861,51.112943,1.742183e+05,...,12.959162,-537789.698593,77.652943,132.443115,105.048029,3.440285,5.109489,100.0,98.540146,13.697543
2025-04-25,WFC,69.73,17775914,71.245217,5.721914,26.811594,89.130435,31.884058,51.813685,2.865265e+05,...,12.297850,-577349.404689,77.149854,132.545508,104.847681,0.641711,2.898551,100.0,92.753623,13.848913
2025-04-28,WFC,69.43,16694076,71.538750,5.372290,NaN,86.764706,29.411765,49.345163,-1.056172e+05,...,11.482734,-591375.952469,76.361517,132.970689,104.666103,0.035651,1.470588,100.0,86.029412,14.152293
2025-04-29,WFC,71.10,24563554,71.636912,5.238784,NaN,93.382353,42.647059,49.510763,-8.045819e+04,...,10.567088,-598296.095588,75.759582,133.181007,104.470294,0.000000,0.735294,100.0,91.176471,14.355356
